In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import SGDClassifier
from sklearn.multioutput import MultiOutputClassifier
import gc

print("--- 🛠️ CHẾ ĐỘ SỬA LỖI KHỚP ID (DEBUG & FIX) ---")

# --- CẤU HÌNH ---
TRAIN_SEQ = '/kaggle/input/cafa-6-protein-function-prediction/Train/train_sequences.fasta'
TRAIN_TERMS = '/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv'
TEST_SEQ = '/kaggle/input/cafa-6-protein-function-prediction/Test/testsuperset.fasta'

NUM_TERMS = 300
AMINO_ACIDS = "ACDEFGHIKLMNPQRSTVWY"

# --- 1. HÀM LÀM SẠCH ID (MẠNH HƠN) ---
def advanced_clean_id(raw_id):
    # Xóa dấu > và khoảng trắng thừa
    raw_id = raw_id.strip().replace('>', '')
    
    # Trường hợp 1: Chuỗi có dấu gạch đứng |
    if '|' in raw_id:
        parts = raw_id.split('|')
        # Thường ID chuẩn nằm ở giữa (index 1)
        if len(parts) >= 2:
            return parts[1]
            
    # Trường hợp 2: Chuỗi có khoảng trắng 
    if ' ' in raw_id:
        return raw_id.split(' ')[0]
        
    # Trường hợp mặc định: Giữ nguyên
    return raw_id

# --- 2. HÀM TẠO ĐẶC TRƯNG ---
def get_aac(sequence):
    seq = sequence.upper()
    length = len(seq)
    if length == 0: return [0.0]*20
    return [seq.count(aa) / length for aa in AMINO_ACIDS]

def load_data_robust(fasta_path):
    print(f"-> Đang đọc file: {os.path.basename(fasta_path)}...")
    ids = []
    features = []
    raw_headers = [] # Lưu lại để debug
    
    with open(fasta_path, 'r') as f:
        cid, cseq = "", []
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                if cid:
                    ids.append(cid); features.append(get_aac("".join(cseq)))
                
                # Lưu header gốc để kiểm tra
                if len(raw_headers) < 5: raw_headers.append(line)
                
                # Áp dụng làm sạch ID
                cid = advanced_clean_id(line)
                cseq = []
            else: cseq.append(line)
        if cid: ids.append(cid); features.append(get_aac("".join(cseq)))
            
    return ids, np.array(features, dtype=np.float32), raw_headers

# --- 3. KIỂM TRA DỮ LIỆU (QUAN TRỌNG) ---
print("[1/4] Đọc và Soi dữ liệu...")

train_ids, X_train, sample_headers = load_data_robust(TRAIN_SEQ)
print(f"   + 5 Header gốc trong Fasta: {sample_headers}")
print(f"   + 5 ID sau khi làm sạch   : {train_ids[:5]}")

print("-> Đang đọc file nhãn...")
train_terms = pd.read_csv(TRAIN_TERMS, sep='\t', header=None, skiprows=1, usecols=[0, 1])
# Đặt tên cột tạm
train_terms.columns = ["EntryID", "term"]

# In mẫu ID trong file Terms
print(f"   + 5 ID gốc trong file Terms: {train_terms['EntryID'].head(5).tolist()}")

# Thử làm sạch ID trong file Terms luôn 
train_terms['EntryID_Clean'] = train_terms['EntryID'].apply(lambda x: advanced_clean_id(str(x)))
print(f"   + 5 ID Terms sau khi clean : {train_terms['EntryID_Clean'].head(5).tolist()}")

# --- 4. KHỚP DỮ LIỆU ---
train_ids_set = set(train_ids)
terms_ids_set = set(train_terms['EntryID_Clean'].unique())
common_ids = train_ids_set.intersection(terms_ids_set)

print(f"\n---> KẾT QUẢ KHỚP: Tìm thấy {len(common_ids)} ID chung.")

if len(common_ids) == 0:
    print("\n❌ VẪN KHÔNG KHỚP! Hãy nhìn kỹ 2 dòng 'ID sau khi làm sạch' ở trên xem chúng khác nhau chỗ nào.")
    print("Ví dụ: Một bên là 'P12345', một bên là 'P12345_HUMAN' -> Khác nhau.")
    raise ValueError("Dừng chương trình để kiểm tra ID.")

# --- 5. NẾU KHỚP THÌ CHẠY TIẾP ---
print("\n✅ Đã khớp ID! Đang tiến hành huấn luyện...")

# Lọc file terms
train_terms = train_terms[train_terms['EntryID_Clean'].isin(common_ids)]

# Lấy Top N
top_terms = train_terms['term'].value_counts().head(NUM_TERMS).index.tolist()
term_to_idx = {t: i for i, t in enumerate(top_terms)}

# Tạo Y
Y_train = np.zeros((len(train_ids), NUM_TERMS), dtype=np.int8)
# Map vị trí ID trong train_ids
id_to_index = {pid: i for i, pid in enumerate(train_ids) if pid in common_ids}

grouped = train_terms.groupby('EntryID_Clean')['term']
for pid, terms in grouped:
    if pid in id_to_index:
        idx_row = id_to_index[pid]
        indices = [term_to_idx[t] for t in terms if t in term_to_idx]
        Y_train[idx_row, indices] = 1

# Loại bỏ cột rỗng
col_sums = Y_train.sum(axis=0)
valid_cols = np.where(col_sums > 0)[0]
Y_train = Y_train[:, valid_cols]
top_terms = [top_terms[i] for i in valid_cols]
NUM_TERMS = len(top_terms)

print(f"-> Train shape: {X_train.shape} | Y shape: {Y_train.shape}")

# Train
clf = MultiOutputClassifier(SGDClassifier(loss='log_loss', alpha=0.001, n_jobs=-1, random_state=42))
clf.fit(X_train, Y_train)
print("-> Training xong!")

# Predict Test
print("[3/4] Dự đoán Test...")
test_ids, X_test, _ = load_data_robust(TEST_SEQ)

if len(test_ids) == 0:
    # Fallback tìm file
    for root, _, files in os.walk('/kaggle/input'):
        for f in files:
            if 'test' in root.lower() and f.endswith('fasta'):
                test_ids, X_test, _ = load_data_robust(os.path.join(root, f))
                break

# Predict từng phần
batch_size = 5000
preds_matrix = np.zeros((len(test_ids), NUM_TERMS), dtype=np.float32)
for i in range(0, len(test_ids), batch_size):
    end = min(i+batch_size, len(test_ids))
    p = clf.predict_proba(X_test[i:end])
    preds_matrix[i:end] = np.array([prob[:, 1] for prob in p]).T

# Ghi file
print("[4/4] Ghi file submission...")
data = []
TOP_K = 30
for i, pid in enumerate(test_ids):
    sc = preds_matrix[i]
    top_ix = np.argsort(sc)[-TOP_K:][::-1]
    for idx in top_ix:
        if sc[idx] > 0.001:
            data.append(f"{pid}\t{top_terms[idx]}\t{sc[idx]:.4f}\n")

# Fallback
if len(data) < len(test_ids):
    print("-> Adding fallback...")
    naive = pd.Series(col_sums, index=top_terms).nlargest(5) / len(Y_train)
    exist = set([x.split('\t')[0] for x in data])
    for pid in test_ids:
        if pid not in exist:
            for t, s in naive.items():
                data.append(f"{pid}\t{t}\t{s:.4f}\n")

with open('submission.tsv', 'w') as f:
    f.writelines(data)
    
print("✅ HOÀN TẤT!")

--- 🛠️ CHẾ ĐỘ SỬA LỖI KHỚP ID (DEBUG & FIX) ---
[1/4] Đọc và Soi dữ liệu...
-> Đang đọc file: train_sequences.fasta...
   + 5 Header gốc trong Fasta: ['>sp|A0A0C5B5G6|MOTSC_HUMAN Mitochondrial-derived peptide MOTS-c OS=Homo sapiens OX=9606 GN=MT-RNR1 PE=1 SV=1', '>sp|A0JNW5|BLT3B_HUMAN Bridge-like lipid transfer protein family member 3B OS=Homo sapiens OX=9606 GN=BLTP3B PE=1 SV=2', '>sp|A0JP26|POTB3_HUMAN POTE ankyrin domain family member B3 OS=Homo sapiens OX=9606 GN=POTEB3 PE=1 SV=2', '>sp|A0PK11|CLRN2_HUMAN Clarin-2 OS=Homo sapiens OX=9606 GN=CLRN2 PE=1 SV=1', '>sp|A1A4S6|RHG10_HUMAN Rho GTPase-activating protein 10 OS=Homo sapiens OX=9606 GN=ARHGAP10 PE=1 SV=1']
   + 5 ID sau khi làm sạch   : ['A0A0C5B5G6', 'A0JNW5', 'A0JP26', 'A0PK11', 'A1A4S6']
-> Đang đọc file nhãn...
   + 5 ID gốc trong file Terms: ['Q5W0B1', 'Q5W0B1', 'Q5W0B1', 'Q5W0B1', 'Q5W0B1']
   + 5 ID Terms sau khi clean : ['Q5W0B1', 'Q5W0B1', 'Q5W0B1', 'Q5W0B1', 'Q5W0B1']

---> KẾT QUẢ KHỚP: Tìm thấy 82404 ID chung.

✅ 